Data is taken from the Thomson Reuters Tick History (TRTH) database. I obtain three months of data (from 1st October to 31st December 2019) for each of the selected stocks. I find three months to be a suitable period of time which ensures that the analysis has statistical power.

The sample comprises the constituents of the S&P/ASX 200, of which only 50 will be incorporated into the analysis, ensuring an adequate mix of Australian stocks with different levels of liquidity. The 50 stocks are selected through randomly using the `sample` command in STATA. The 50 stocks selected are:

|Code|Company|Code|Company|Code|Company
|---|---|---|---|---|---|
|APX|Appen Limited|DXS|Dexus Units FP Stapled|ORA|Orora Ltd
|BEN|Bendigo and Adelaide Bank|INA|Ingenia Group|GMG|Goodman Group
|BHP|BHP Billiton Ltd|CUV|Clinuvel Pharmaceut|NHF|Nib Holdings Limited
|BKW|Brickworks Ltd|ALL|Aristocrat Leisure|EVN|Evolution Mining Ltd
|SPK|Spark New Zealand|SGR|The Star Ent Group|MQG|Macquarie Group Ltd
|TLS|Telstra Corporation|RSG|Resolute Mining|AVH|Avita Medical
|BOQ|Bank of Queensland|SCG|Scentre Group|PNI|Pinnacle Investment
|URW|Unibail-Rodamco-Westfield|QBE|QBE Insurance Group|MTS|Metcash Ltd
|PMV|Premier Investments|PLS|Pilbara Min Limited|SAR|Saracen Mineral
|WEB|Webjet Ltd|IFL|IOOF Holdings Ltd|XRO|Xero Ltd
|DHG|Domain Holdings Australia|BIN|Bingo Industries Limited|CQR|Charter Hall Retail
|DOW|Downer Edi Ltd|FMG|Fortescue Metals Group|NUF|Nufarm
|HUB|HUB24 Ltd|FLT|Flight Centre Travel|BPT|Beach Energy Ltd
|PPT|Perpetual Ltd|MFG|Magellan Fin Group Ltd|BXB|Brambles Ltd
|SLR|Silver Lake Resource|RMD|Resmed Inc|CCP|Credit Corp Group
|ELD|Elders Limited|JHX|James Hardie Industries|AMC|Amcor Ltd
|ASB|Austral Ltd|ALU|Altium Ltd|||
			
			
			
			
			
			
			
		
		
		
		



<font color=Red>

## Data Cleaning

</font>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Changing format in which numbers are displayed
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
df = pd.read_csv('asx_consolidated.csv')

In [3]:
df.head()

,#RIC,Domain,Date-Time,Type,Ex/Cntrb.ID,Price,Volume,Buyer ID,Seller ID,Qualifiers,Date,Unique Trade Identification
0,ALL.AUX,Market Price,2019-10-01T05:15:00.359179688+10,Trade,NaN,NaN,NaN,NaN,NaN,[PRC_QL_CD]; [PRC_QL2];[GV4_TEXT],NaN,NaN
1,ALL.AUX,Market Price,2019-10-01T09:59:51.430531511+10,Trade,ASX,30.85,123.0,ASX,ASX,MOXT[GV4_TEXT],NaN,NaN
2,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,150.0,NaN,NaN,MO[GV4_TEXT],NaN,NaN
3,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,284.0,NaN,NaN,MO[GV4_TEXT],NaN,NaN
4,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,185.0,NaN,NaN,MO[GV4_TEXT],NaN,NaN


There are some cells in dataframe which have values displayed as `NaN` ; this means they are missing values and the variable is not relevant for our analysis, so we can proceed to remove these columns.

In [5]:
df = df.drop(['Buyer ID', 'Seller ID', 'Unique Trade Identification', 'Date'], axis=1)

In [6]:
df.head(5)

,#RIC,Domain,Date-Time,Type,Ex/Cntrb.ID,Price,Volume,Qualifiers
0,ALL.AUX,Market Price,2019-10-01T05:15:00.359179688+10,Trade,NaN,NaN,NaN,[PRC_QL_CD]; [PRC_QL2];[GV4_TEXT]
1,ALL.AUX,Market Price,2019-10-01T09:59:51.430531511+10,Trade,ASX,30.85,123.0,MOXT[GV4_TEXT]
2,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,150.0,MO[GV4_TEXT]
3,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,284.0,MO[GV4_TEXT]
4,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,185.0,MO[GV4_TEXT]


Next, I look at observations which do not contain values for either price and/or volume. These observations are likely initialising observations for the day and are not relevant to the analysis.

In [7]:
df.dropna(axis=0, inplace=True)

In [8]:
df.head()

,#RIC,Domain,Date-Time,Type,Ex/Cntrb.ID,Price,Volume,Qualifiers
1,ALL.AUX,Market Price,2019-10-01T09:59:51.430531511+10,Trade,ASX,30.85,123.0,MOXT[GV4_TEXT]
2,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,150.0,MO[GV4_TEXT]
3,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,284.0,MO[GV4_TEXT]
4,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,185.0,MO[GV4_TEXT]
5,ALL.AUX,Market Price,2019-10-01T09:59:51.431464169+10,Trade,ASX,30.85,33.0,MO[GV4_TEXT]


Since all stocks I am working with here are pulled from consolidated data, I remove the ".AUX" from each of the stocks in the variable `#RIC`.

In [9]:
df['#RIC'] = df['#RIC'].map(lambda x: x[:-4])
df.head()

To make sure that the values stored in each column are in their correct format (i.e. `Price` should be numerical, `Qualifiers` should be string/object, `Date-Time` should be in DateTime format), we can use the `info` method:

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30950255 entries, 1 to 30953459
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   #RIC         object        
 1   Domain       object        
 2   Date-Time    datetime64[ns]
 3   Type         object        
 4   Ex/Cntrb.ID  object        
 5   Price        float64       
 6   Volume       float64       
 7   Qualifiers   object        
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 2.1+ GB


It seems like everything is fine except `Date-Time`. Python stores the category `Date-Time` as a series of `object`s (strings) rather than dates. I can parse it so that it is recognised in an actual `datetime` format, allowing me to interact with it in more meaningful ways.

First, we can remove the timezone from the string.

In [11]:
df['Date-Time'] = df['Date-Time'].map(lambda x: x[:-3])
df.head()

,#RIC,Domain,Date-Time,Type,Ex/Cntrb.ID,Price,Volume,Qualifiers
1,ALL,Market Price,2019-10-01T09:59:51.430531511,Trade,ASX,30.85,123.0,MOXT[GV4_TEXT]
2,ALL,Market Price,2019-10-01T09:59:51.431464169,Trade,ASX,30.85,150.0,MO[GV4_TEXT]
3,ALL,Market Price,2019-10-01T09:59:51.431464169,Trade,ASX,30.85,284.0,MO[GV4_TEXT]
4,ALL,Market Price,2019-10-01T09:59:51.431464169,Trade,ASX,30.85,185.0,MO[GV4_TEXT]
5,ALL,Market Price,2019-10-01T09:59:51.431464169,Trade,ASX,30.85,33.0,MO[GV4_TEXT]


Now we can proceed to convert the string to `datetime` format.

In [12]:
df['Date-Time'] = pd.to_datetime(df['Date-Time'], format="%Y-%m-%dT%H:%M:%S.%f")

8.75 s ± 1.49 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
df['Date-Time'].dtype

dtype('<M8[ns]')

We can see the variable `Date-Time` is now being recognised as a `datetime` datatype ( `datetime64[ns]` maps to `<M8[ns]` ).

The exported dataset has its `Qualifier` variable displayed in a way which describes the type of trade/crossing, followed by “[GV4_TEXT]”. A qualifier is an identifier which provides some additional information relating to the trade (e.g. whether it originated from a dark pool, crossing trades etc.) We can use a dictionary to map these values to a more useful description:

In [14]:
dictqual = {'ACXT[GV4_TEXT]' : 'auct_close',
            'AC[GV4_TEXT]' : 'auct_close',
            'IIBB[GV4_TEXT]' : 'omh', #on market hidden cross-trades
            'IIB[GV4_TEXT]' : 'omh', #on market hidden trades
            'MOXT[GV4_TEXT]' : 'auct_open',
            'MO[GV4_TEXT]' : 'auct_open',
            'NXXT[GV4_TEXT]' : 'nbbo',#broker dark trades (incl. manual executions) - reported to ASX
            'S[GV4_TEXT]' : 'nbbo', #broker dark trades (incl. manual executions) - reported to Chi-X
            'XT[GV4_TEXT]' : 'oml', #on market lit cross-trades
            '[GV4_TEXT]' : 'oml', #on market lit trades
            'CX[GV4_TEXT]' : 'cp', #Centre Point cross-trade
            'CPXT[GV4_TEXT]' : 'cppref', #CP preference-matched cross-trade
            'CTXT[GV4_TEXT]' : 'cpoth',
            'CXTU[GV4_TEXT]' : 'cpoth',
            'CXTP[GV4_TEXT]' : 'cpprefprior',
            'S1XT[GV4_TEXT]' : 'block', #block trades
            'S2XT[GV4_TEXT]' : 'block',
            'S3XT[GV4_TEXT]' : 'block',
            'SXXT[GV4_TEXT]' : 'block',
            'SPXT[GV4_TEXT]' : 'block',
            'P1XT[GV4_TEXT' : 'block',
            'B[GV4_TEXT]' : 'block'} #block trades on Chi-X

In [18]:
df.replace({"Qualifiers": dictqual}, inplace=True)

In [25]:
df.head(5)

,#RIC,Domain,Date-Time,Type,Ex/Cntrb.ID,Price,Volume,Qualifiers
1,ALL,Market Price,2019-10-01 09:59:51.430531511,Trade,ASX,30.85,123.0,auct_open
2,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.85,150.0,auct_open
3,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.85,284.0,auct_open
4,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.85,185.0,auct_open
5,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.85,33.0,auct_open


In order to get the data ready for computation of descriptive statistics, I need to add a few variables. First is `Value`, which is simply `Price` multiplied by `Volume`; this gives us the value of each trade. I also need a variable `Trade Type` which categorises a particular trade into one of the three groups: Centre Point trade `cp`, Broker-operated dark pool trade `b` and CLOB trade `clob`; the categorisation is based on the qualifiers, described above. Finally, I add `Month`, which saves me having to group by month later on. 

In [48]:
# Define category of each qualifier
clob = ["omh", "oml", "auct_close", "auct_open"]
cp = ["cp", "cppref", "cpoth", "cpprefprior"]
b = ["nbbo"]

def trade_type_variable(df):  
    if df['Qualifiers'] in clob:
        return "clob"
    elif df['Qualifiers'] in cp:
        return "cp"
    elif df['Qualifiers'] in b:
        return "b"
    else:
        return None

df['Trade Type'] = df.apply(trade_type_variable, axis=1)
df.head()

,#RIC,Domain,Date-Time,Type,Ex/Cntrb.ID,Price,Volume,Qualifiers,Trade Type
1,ALL,Market Price,2019-10-01 09:59:51.430531511,Trade,ASX,30.850,123.000,auct_open,clob
2,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.850,150.000,auct_open,clob
3,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.850,284.000,auct_open,clob
4,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.850,185.000,auct_open,clob
5,ALL,Market Price,2019-10-01 09:59:51.431464169,Trade,ASX,30.850,33.000,auct_open,clob


In [49]:
# Create a new variable "Value"
df['Value'] = df.apply(lambda row: row["Price"] * row["Volume"] , axis=1)

In [58]:
# Create a new variable "Month"
df['Month'] = df['Date-Time'].dt.month
# Changing dtype from int to str
df["Month"] = df["Month"].astype(str)
# Replace month number with word
df["Month"].replace({"10": "October", "11": "November", "12": "December"}, inplace=True)

In [59]:
# Rearrange variable columns
df = df[["#RIC", "Domain", "Date-Time", "Month", "Type", "Ex/Cntrb.ID", "Price", "Volume", "Value", "Qualifiers", "Trade Type"]]
df.head()

,#RIC,Domain,Date-Time,Month,Type,Ex/Cntrb.ID,Price,Volume,Value,Qualifiers,Trade Type
1,ALL,Market Price,2019-10-01 09:59:51.430531511,October,Trade,ASX,30.850,123.000,3794.550,auct_open,clob
2,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,150.000,4627.500,auct_open,clob
3,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,284.000,8761.400,auct_open,clob
4,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,185.000,5707.250,auct_open,clob
5,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,33.000,1018.050,auct_open,clob


In order to create summary statistics (which need to be grouped by month) more easily, I split the dataframe into three different ones, each containing trades of one of the three trade types (CP, broker dark pools and CLOB).

In [60]:
# Dataframe for CLOB trades
clob = df[df['Trade Type'] == 'clob']
clob.head()

,#RIC,Domain,Date-Time,Month,Type,Ex/Cntrb.ID,Price,Volume,Value,Qualifiers,Trade Type
1,ALL,Market Price,2019-10-01 09:59:51.430531511,October,Trade,ASX,30.850,123.000,3794.550,auct_open,clob
2,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,150.000,4627.500,auct_open,clob
3,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,284.000,8761.400,auct_open,clob
4,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,185.000,5707.250,auct_open,clob
5,ALL,Market Price,2019-10-01 09:59:51.431464169,October,Trade,ASX,30.850,33.000,1018.050,auct_open,clob


In [61]:
# Dataframe for broker dark trades
b = df[df['Trade Type'] == 'b']
b.head()

,#RIC,Domain,Date-Time,Month,Type,Ex/Cntrb.ID,Price,Volume,Value,Qualifiers,Trade Type
124,ALL,Market Price,2019-10-01 10:00:06.578631677,October,Trade,CHA,30.740,30.000,922.200,nbbo,b
136,ALL,Market Price,2019-10-01 10:00:16.129812013,October,Trade,ASX,30.770,13.000,400.010,nbbo,b
225,ALL,Market Price,2019-10-01 10:01:07.186767200,October,Trade,ASX,30.895,10.000,308.950,nbbo,b
287,ALL,Market Price,2019-10-01 10:01:21.274107839,October,Trade,ASX,30.935,14.000,433.090,nbbo,b
305,ALL,Market Price,2019-10-01 10:01:27.137375310,October,Trade,CHA,30.940,36.000,1113.840,nbbo,b


In [62]:
# Dataframe for Centre Point trades
cp = df[df['Trade Type'] == 'cp']
cp.head()

,#RIC,Domain,Date-Time,Month,Type,Ex/Cntrb.ID,Price,Volume,Value,Qualifiers,Trade Type
117,ALL,Market Price,2019-10-01 09:59:54.250673237,October,Trade,ASX,30.730,27.000,829.710,cp,cp
162,ALL,Market Price,2019-10-01 10:01:05.130282501,October,Trade,ASX,30.830,17.000,524.110,cp,cp
164,ALL,Market Price,2019-10-01 10:01:05.142613959,October,Trade,ASX,30.830,75.000,2312.250,cp,cp
199,ALL,Market Price,2019-10-01 10:01:05.810346037,October,Trade,ASX,30.885,100.000,3088.500,cp,cp
234,ALL,Market Price,2019-10-01 10:01:09.685570311,October,Trade,ASX,30.875,57.000,1759.875,cp,cp


<font color=Red>

## Summary Statistics

</font>

<font color=Blue>

### Summary statistics on the level of trading activity in the Centre Point dark pool

</font>

In [74]:
cp.groupby('Month').agg(
    Mean_Trade_Size = pd.NamedAgg(column = "Volume", aggfunc="mean"),
    Median_Trade_Size = pd.NamedAgg(column = "Volume", aggfunc="median"),
    Mean_Trade_Value = pd.NamedAgg(column = "Value", aggfunc="mean"),
    Median_Trade_Value = pd.NamedAgg(column = "Value", aggfunc="mean"),
    Average_Daily_Volume = pd.NamedAgg(column = "Volume", aggfunc=lambda x: sum(x) / 30),
    Average_Daily_Trade_Value = pd.NamedAgg(column = "Value", aggfunc=lambda x: sum(x) / 30)
    
    # Still figuring out how to compute....
    #     Percentage of total volume
    #     Percentage of total value
    
).transpose()

Month,December,November,October
Mean_Trade_Size,305.402,330.806,343.021
Median_Trade_Size,18.000,21.000,21.000
Mean_Trade_Value,2138.447,2419.746,2261.234
Median_Trade_Value,2138.447,2419.746,2261.234
Average_Daily_Volume,9272796.600,11211451.867,11737298.133
Average_Daily_Trade_Value,64928814.377,82008344.473,77373536.398


<font color=Blue>

### Summary statistics on the level of trading activity in broker-operated dark pools

</font>

In [75]:
b.groupby('Month').agg(
    Mean_Trade_Size = pd.NamedAgg(column = "Volume", aggfunc="mean"),
    Median_Trade_Size = pd.NamedAgg(column = "Volume", aggfunc="median"),
    Mean_Trade_Value = pd.NamedAgg(column = "Value", aggfunc="mean"),
    Median_Trade_Value = pd.NamedAgg(column = "Value", aggfunc="mean"),
    Average_Daily_Volume = pd.NamedAgg(column = "Volume", aggfunc=lambda x: sum(x) / 30),
    Average_Daily_Trade_Value = pd.NamedAgg(column = "Value", aggfunc=lambda x: sum(x) / 30)
    
    # Still figuring out how to compute....
    #     Percentage of total volume
    #     Percentage of total value
    
).transpose()

Month,December,November,October
Mean_Trade_Size,249.696,249.924,254.688
Median_Trade_Size,18.000,20.000,20.000
Mean_Trade_Value,1605.983,1706.742,1640.358
Median_Trade_Value,1605.983,1706.742,1640.358
Average_Daily_Volume,5695986.933,6767832.467,7342369.000
Average_Daily_Trade_Value,36635199.836,46217767.697,47289606.506


<font color=Blue>

### Summary statistics on the level of trading activity on the CLOB

</font>

In [76]:
clob.groupby('Month').agg(
    Mean_Trade_Size = pd.NamedAgg(column = "Volume", aggfunc="mean"),
    Median_Trade_Size = pd.NamedAgg(column = "Volume", aggfunc="median"),
    Mean_Trade_Value = pd.NamedAgg(column = "Value", aggfunc="mean"),
    Median_Trade_Value = pd.NamedAgg(column = "Value", aggfunc="mean"),
    Average_Daily_Volume = pd.NamedAgg(column = "Volume", aggfunc=lambda x: sum(x) / 30),
    Average_Daily_Trade_Value = pd.NamedAgg(column = "Value", aggfunc=lambda x: sum(x) / 30)
    
    # Still figuring out how to compute....
    #     Percentage of total volume
    #     Percentage of total value
    
).transpose()

Month,December,November,October
Mean_Trade_Size,453.284,427.475,399.517
Median_Trade_Size,34.000,34.000,36.000
Mean_Trade_Value,3238.741,3038.342,2789.107
Median_Trade_Value,3238.741,3038.342,2789.107
Average_Daily_Volume,108570815.367,125202335.767,128340463.867
Average_Daily_Trade_Value,775745495.650,889893365.891,895971178.675
